In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import getpass

plt.rcParams['axes.prop_cycle'] = plt.cycler(color = sns.color_palette('dark'))
plt.style.use('dark_background')
plt.rcParams['figure.figsize'] = (10 , 8)

In [ ]:
from openai import OpenAI

# os.environ['OPENAI_API_KEY'] = 'none'

client = OpenAI()
print('OpenAI client connected successfully!')

OpenAI client connected successfully!


In [6]:
def loan_decision_with_generation_model(applicant_data):
    """
    Using GPT-4o-mini (generation architecture)
    Requires explicit decomposition of the decision process
    """

    structured_prompt = f"""
You are evaluating a loan application. Follow these exact steps:

STEP 1 - Compute debt-to-income ratio:
Monthly debt: ${applicant_data['debt']}
Monthly income: ${applicant_data['income']}
Formula: debt ÷ income
Your calculation: _____

STEP 2 - Classify credit tier:
Score: {applicant_data['credit_score']}
Tiers:
  740+: Premium
  670-739: Standard
  580-669: Subprime
  <580: High-risk
Classification: _____

STEP 3 - Verify employment:
Months employed: {applicant_data['employment_months']}
Minimum required: 24 months
Income stability (variance): {applicant_data['income_variance']}%
Maximum allowed: 15%
Status: _____

STEP 4 - Assess collateral:
Property value: ${applicant_data['collateral']}
Loan amount: ${applicant_data['loan_amount']}
LTV ratio: _____

[... would continue for all decision criteria ...]

FINAL OUTPUT: Approved/Denied with justification
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": structured_prompt}],
        temperature=0
    )

    return response.choices[0].message.content


def loan_decision_with_reasoning_model(applicant_data):
    """
    Using o1-mini (reasoning architecture)
    Model performs internal decomposition automatically
    """

    minimal_prompt = f"""
Evaluate this loan application against our criteria:

Applicant Data:
- Monthly debt: ${applicant_data['debt']}
- Monthly income: ${applicant_data['income']}
- Credit score: {applicant_data['credit_score']}
- Employment: {applicant_data['employment_months']} months
- Income variance: {applicant_data['income_variance']}%
- Collateral value: ${applicant_data['collateral']}
- Requested amount: ${applicant_data['loan_amount']}

Approval Criteria:
- Debt-to-income < 43%
- Credit score ≥ 580
- Employment ≥ 24 months with income variance < 15%
- Adequate collateral coverage

Decision: Approve or deny? Explain your reasoning.
"""

    response = client.chat.completions.create(
        model="o1",
        messages=[{"role": "user", "content": minimal_prompt}]
    )

    return response.choices[0].message.content


# Example applicant
test_application = {
    'debt': 2800,
    'income': 7500,
    'credit_score': 695,
    'employment_months': 30,
    'income_variance': 12.5,
    'collateral': 180000,
    'loan_amount': 140000
}

print("="*70)
print("GENERATION MODEL (GPT-4o-mini)")
print("="*70)
print("Prompt strategy: Explicit step-by-step scaffolding")
print("Engineer's role: Design the reasoning chain")
print("Token cost: Lower per call")
print()

print("="*70)
print("REASONING MODEL (o1-mini)")
print("="*70)
print("Prompt strategy: State goal + constraints")
print("Engineer's role: Define success criteria")
print("Token cost: Higher per call, but more reliable logic")
print("="*70)

GENERATION MODEL (GPT-4o-mini)
Prompt strategy: Explicit step-by-step scaffolding
Engineer's role: Design the reasoning chain
Token cost: Lower per call

REASONING MODEL (o1-mini)
Prompt strategy: State goal + constraints
Engineer's role: Define success criteria
Token cost: Higher per call, but more reliable logic


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

/opt/anaconda3/envs/torch/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
prompt = 'What is the capital of France ?'
tokens = tokenizer.encode(prompt , return_tensors = 'pt')
tokens

tensor([[2061,  318,  262, 3139,  286, 4881, 5633]])

In [6]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [10]:
output = model.generate(
    tokens , max_new_tokens = 50,
    do_sample = True,
    temperature = 0.7,
    pad_token_id = tokenizer.eos_token_id
)
print(f"{model_name} output: {tokenizer.decode(output[0] , skip_special_tokens= True)}")

gpt2 output: What is the capital of France ? It is the capital of France . . . We do not say that the capital of France is French."

"The capital of France is French. It is the capital of France . . . We do not say that the capital of France is


In [11]:
prompts = [
    "Translate 'Hello' to French:",
    "Write a poem about the ocean:",
    "Summarize the following: The quick brown fox jumps over the lazy dog.",
    "Answer the following question: What is 2+2?",
]

for prompt in prompts:
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(
        input_ids,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id
    )
    print(f"Prompt: {prompt}")
    print(f"Output: {tokenizer.decode(output[0], skip_special_tokens=True)}\n")
    print('-'*50)

Prompt: Translate 'Hello' to French:
Output: Translate 'Hello' to French:

The German translation is:

Hello!

This is a special version of the German version.

The English translation is:

--------------------------------------------------
Prompt: Write a poem about the ocean:
Output: Write a poem about the ocean: "I was living through a wave of a global tsunami. I was watching a lot of things happening in the ocean. I heard the waves and thought

--------------------------------------------------
Prompt: Summarize the following: The quick brown fox jumps over the lazy dog.
Output: Summarize the following: The quick brown fox jumps over the lazy dog.

The quick brown fox jumps over the lazy dog. The quick fox jumps over the lazy dog. The quick fox jumps over the lazy dog.

--------------------------------------------------
Prompt: Answer the following question: What is 2+2?
Output: Answer the following question: What is 2+2?

Answer: The 2+2 factor is the number of points to give to a

## Instruction Model:


In [2]:
from transformers import pipeline
import torch

# Load an instruct model
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
generator = pipeline(
    "text-generation",
    model=model_name,
    dtype=torch.float16,
    device_map="auto"
)

# Same prompt as before - using instruction format
prompt = "What is the capital of France?"
formatted_prompt = f"<|user|>\n{prompt}</s>\n<|assistant|>\n"

output = generator(
    formatted_prompt,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
)

Device set to use mps


In [12]:
print(f"{prompt}")
print(f"Response: {output[0]['generated_text'].split('<|assistant|>')[-1].strip()}")

What is the capital of France?
Response: Yes, the capital of France is Paris. However, the official name of the city as well as the seat of the country's government is the City of Paris. The city of Paris is divided into five arrondissements (districts


In [22]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of Austria?"},
]
output = generator(messages , max_new_tokens = 50)
print(f"{output[0]['generated_text'][-1]['content']}")

The capital of Austria is Vienna.


In [24]:
# Multi-turn conversation demonstration
messages = [
    {"role": "system", "content": "You are a helpful math tutor."},
    {"role": "user", "content": "What is 5 + 3?"},
]

# First turn
output = generator(messages, max_new_tokens=50)
assistant_response = output[0]['generated_text'][-1]['content']
print(f"User: What is 5 + 3?")
print(f"Assistant: {assistant_response}\n")

# Add assistant response to history
messages.append({"role": "assistant", "content": assistant_response})

# Second turn - referring to previous context
messages.append({"role": "user", "content": "Now multiply that result by 2."})
output = generator(messages, max_new_tokens=50)
assistant_response = output[0]['generated_text'][-1]['content']
print(f"User: Now multiply that result by 2.")
print(f"Assistant: {assistant_response}\n")

# Third turn
messages.append({"role": "assistant", "content": assistant_response})
messages.append({"role": "user", "content": "What if I divide that by 4?"})
output = generator(messages, max_new_tokens=50)
assistant_response = output[0]['generated_text'][-1]['content']
print(f"User: What if I divide that by 4?")
print(f"Assistant: {assistant_response}")

User: What is 5 + 3?
Assistant: Yes, I can help you with that.

5 + 3 = 8

The expression "5 + 3" represents addition of 5 and 3, which equals 8.

User: Now multiply that result by 2.
Assistant: Yes, I can help you with that.

8 * 2 = 16

The expression "8 * 2" represents multiplication of 8 and 2, which equals 16.

User: What if I divide that by 4?
Assistant: Sure, I can help you with that.

16 / 4 = 4

The expression "16 / 4" represents division of 16 by 4, which equals 4.
